# Modelling

In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('Data/clean_data.csv')

df

,sentiment,raw_comm,nouns_comm
0,neutral,Technopolis plans to develop in stages an area...,Technopolis stages area meters order companies...
1,negative,The international electronic industry company ...,industry company tens employees facility layof...
2,positive,With the new production plant the company woul...,production plant company capacity increase dem...
3,positive,According to the company 's updated strategy f...,company strategy years term sales growth range...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,FINANCING growth strategy circuit boards PCBs
...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,Share prices rebound bank stocks weakness
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,beer sales per cent litres beer sales cent litres
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,profit mn mn vessel sales gain mn
4843,negative,Net sales of the Paper segment decreased to EU...,sales segment mn quarter mn quarter profit ite...


### Split the data

In [56]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)

# Display the class distribution in the training set before downsampling
print("Training set class distribution before downsampling:")
print(train_df['sentiment'].value_counts())

# Display the class distribution in the test set
print("Test set class distribution:")
print(test_df['sentiment'].value_counts())

Training set class distribution before downsampling:
neutral     2302
positive    1091
negative     483
Name: sentiment, dtype: int64
Test set class distribution:
neutral     576
positive    272
negative    121
Name: sentiment, dtype: int64


### Down sampling

In [191]:
# Determine the size of the smallest class in the training set
min_class_size = train_df['sentiment'].value_counts().max()

# Downsample each sentiment class in the training set
train_positive = train_df[train_df['sentiment'] == 'positive'].sample(min_class_size, replace=True, random_state=42)
train_negative = train_df[train_df['sentiment'] == 'negative'].sample(min_class_size, replace=True, random_state=42)
train_neutral = train_df[train_df['sentiment'] == 'neutral'].sample(min_class_size, replace=True, random_state=42)

# Combine the downsampled dataframes
train_df_downsampled = pd.concat([train_positive, train_negative, train_neutral])

# Shuffle the combined dataframe to mix the classes
train_df_downsampled = train_df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the class distribution after downsampling in the training set
print("Training set class distribution after downsampling:")
print(train_df_downsampled['sentiment'].value_counts())


Training set class distribution after downsampling:
negative    2302
positive    2302
neutral     2302
Name: sentiment, dtype: int64


### TF-IDF Vectorization

In [192]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fill NaN values in 'nouns_comm' and 'raw_comm' columns with an empty string
train_df_downsampled['nouns_comm'] = train_df_downsampled['nouns_comm'].fillna('')
train_df_downsampled['raw_comm'] = train_df_downsampled['raw_comm'].fillna('')
test_df['nouns_comm'] = test_df['nouns_comm'].fillna('')
test_df['raw_comm'] = test_df['raw_comm'].fillna('')

# TF-IDF Vectorization for 'nouns_comm' and 'raw_comm'
tfidf_vectorizer_nouns = TfidfVectorizer()
tfidf_vectorizer_raw = TfidfVectorizer()

X_train_nouns = tfidf_vectorizer_nouns.fit_transform(train_df_downsampled['nouns_comm'])
X_test_nouns = tfidf_vectorizer_nouns.transform(test_df['nouns_comm'])

X_train_raw = tfidf_vectorizer_raw.fit_transform(train_df_downsampled['raw_comm'])
X_test_raw = tfidf_vectorizer_raw.transform(test_df['raw_comm'])

y_train = train_df_downsampled['sentiment']
y_test = test_df['sentiment']


## Naive Bayes Model

### Train the model

In [193]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Train Naive Bayes models for both 'nouns_comm' and 'raw_comm'
nb_nouns = MultinomialNB()
nb_raw = MultinomialNB()   

# Fit the models
nb_nouns.fit(X_train_nouns, y_train)
nb_raw.fit(X_train_raw, y_train)

# Predict on the training set
y_train_pred_nouns = nb_nouns.predict(X_train_nouns)
y_train_pred_raw = nb_raw.predict(X_train_raw)

# Predict on the test set
y_test_pred_nouns = nb_nouns.predict(X_test_nouns)
y_test_pred_raw = nb_raw.predict(X_test_raw)


### Compute models' accuracy

In [194]:
# Evaluate the Naive Bayes model on the test set ('nouns_comm')
print("\nTest Set - Naive Bayes Model on 'nouns_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_nouns))
print("Classification Report:\n", classification_report(y_test, y_test_pred_nouns))

# Evaluate the Naive Bayes model on the test set ('raw_comm')
print("\nTest Set - Naive Bayes Model on 'raw_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_raw))
print("Classification Report:\n", classification_report(y_test, y_test_pred_raw))



Test Set - Naive Bayes Model on 'nouns_comm'
Accuracy: 0.5830753353973168
Classification Report:
               precision    recall  f1-score   support

    negative       0.33      0.65      0.44       121
     neutral       0.79      0.64      0.71       576
    positive       0.45      0.43      0.44       272

    accuracy                           0.58       969
   macro avg       0.52      0.57      0.53       969
weighted avg       0.64      0.58      0.60       969


Test Set - Naive Bayes Model on 'raw_comm'
Accuracy: 0.6800825593395253
Classification Report:
               precision    recall  f1-score   support

    negative       0.44      0.80      0.57       121
     neutral       0.83      0.73      0.78       576
    positive       0.57      0.51      0.54       272

    accuracy                           0.68       969
   macro avg       0.62      0.68      0.63       969
weighted avg       0.71      0.68      0.69       969



## FNN